In [54]:
# initialising libraries
library(tidyverse)
library(rvest)
library(readr)
library(skimr)
library(visdat)
library(purrr)
library(stringr)
library(glue)
library(jsonlite)
library(httr)

In [2]:
# setting up our API query for Auckland Transport
api_query <- 'https://services2.arcgis.com/JkPEgZJGxhSjYOo0/arcgis/rest/services/TrafficService/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'

In [84]:
traffic_raw <- fromJSON(api_query, flatten = TRUE)

traffic_raw %>%
    glimpse()

List of 8
 $ objectIdFieldName    : chr "OBJECTID"
 $ uniqueIdField        :List of 2
  ..$ name              : chr "OBJECTID"
  ..$ isSystemMaintained: logi TRUE
 $ globalIdFieldName    : chr ""
 $ geometryType         : chr "esriGeometryPoint"
 $ spatialReference     :List of 2
  ..$ wkid      : int 4326
  ..$ latestWkid: int 4326
 $ fields               :'data.frame':	22 obs. of  7 variables:
  ..$ name        : chr [1:22] "OBJECTID" "carr_way_no" "road_id" "road_name" ...
  ..$ type        : chr [1:22] "esriFieldTypeOID" "esriFieldTypeInteger" "esriFieldTypeInteger" "esriFieldTypeString" ...
  ..$ alias       : chr [1:22] "OBJECTID" "CARRIAGEWAY NO" "Road ID" "Road Name" ...
  ..$ sqlType     : chr [1:22] "sqlTypeOther" "sqlTypeOther" "sqlTypeOther" "sqlTypeOther" ...
  ..$ domain      : logi [1:22] NA NA NA NA NA NA ...
  ..$ defaultValue: logi [1:22] NA NA NA NA NA NA ...
  ..$ length      : int [1:22] NA NA NA 100 100 100 NA 10 8 5 ...
 $ exceededTransferLimit: logi TRUE
 $ feat

In [91]:
# two elements of interest, fields and features 
# fields seems to be a dictionary of sorts, and features has the actual data
# turning these into dataframes
traffic_fields <- as.data.frame(traffic_raw$fields)
traffic_data <- as.data.frame(traffic_raw$features)
traffic_data %>%
    glimpse()

Rows: 1,000
Columns: 24
$ attributes.OBJECTID       <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1~
$ attributes.carr_way_no    <int> 23788, 34813, 58, 1826, 1627, 1768, 133, 151~
$ attributes.road_id        <int> 50888, 70028, 70028, 70028, 70028, 70028, 70~
$ attributes.road_name      <chr> "BRAMLEY DR", "AWHITU RD", "AWHITU RD", "AWH~
$ attributes.start_name     <chr> "FIELDING CRES (WEST)", "KING ST (SUMPS RHS)~
$ attributes.end_name       <chr> "FIELDING CRES (EAST)", "MILLBROOK DR", "CRA~
$ attributes.location       <int> 522, 230, 5830, 6522, 23199, 26056, 30814, 5~
$ attributes.latest         <chr> "Yes", "Yes", "Yes", "Yes", "Yes", "Yes", "Y~
$ attributes.count_date     <dbl> 1.016669e+12, 1.332374e+12, 1.086912e+12, 6.~
$ attributes.peak_hour      <chr> NA, "1700", "08:00", NA, NA, NA, "1300", "15~
$ attributes.count_duration <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, ~
$ attributes.adt            <int> 451, 3831, 613, 1380, 858, 844, 437, 242, 37~
$ attributes.pea

In [96]:
# the column names of traffic_data could be more readable, currently they are 'attributes.OBJECTID, attributes.carr_way_no' etc. 
# luckily we have the alias column in traffic_fields, let's use that
colnames(traffic_data) <- (traffic_fields$alias) 

traffic_data %>%
    glimpse()


Rows: 1,000
Columns: 24
$ OBJECTID                 <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14~
$ `CARRIAGEWAY NO`         <int> 23788, 34813, 58, 1826, 1627, 1768, 133, 1512~
$ `Road ID`                <int> 50888, 70028, 70028, 70028, 70028, 70028, 700~
$ `Road Name`              <chr> "BRAMLEY DR", "AWHITU RD", "AWHITU RD", "AWHI~
$ `Carriageway Start Name` <chr> "FIELDING CRES (WEST)", "KING ST (SUMPS RHS)"~
$ `Carriageway End Name`   <chr> "FIELDING CRES (EAST)", "MILLBROOK DR", "CRAI~
$ Location                 <int> 522, 230, 5830, 6522, 23199, 26056, 30814, 50~
$ `Latest Count`           <chr> "Yes", "Yes", "Yes", "Yes", "Yes", "Yes", "Ye~
$ `Count Date`             <dbl> 1.016669e+12, 1.332374e+12, 1.086912e+12, 6.4~
$ `Peak Hour`              <chr> NA, "1700", "08:00", NA, NA, NA, "1300", "15:~
$ `Count Duration`         <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N~
$ ADT                      <int> 451, 3831, 613, 1380, 858, 844, 437, 242, 37,~
$ `Peak Traffic`

In [ ]:
# need to change last two column names, 
# make  'Count Date' read as date instead of dbl,
# standardise 'Peak Hour',
# change NA values to proper missing values